# Week 1 Lab: Data Collection for Machine Learning

**CS 203: Software Tools and Techniques for AI**

---

## Lab Overview

In this lab, you will learn to collect data from the web using:

1. **HTTP fundamentals** - Understanding how the web works
2. **curl** - Command-line HTTP client
3. **Python requests** - Programmatic API calls
4. **BeautifulSoup** - Web scraping when APIs don't exist

**Goal**: Build a movie data collection pipeline for Netflix-style movie prediction.

---

## Setup

First, let's install and import the required libraries.

In [1]:
# Install required packages (uncomment if needed)
# !pip install requests beautifulsoup4 pandas

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import time

print("All imports successful!")

All imports successful!


---

# Part 1: HTTP Fundamentals

Before we start collecting data, we need to understand how the web works.

## 1.1 Understanding URLs

A URL (Uniform Resource Locator) has several components:

```
https://api.omdbapi.com:443/v1/movies?t=Inception&y=2010#details
└─┬──┘└──────┬──────┘└┬─┘└──┬───┘└────────┬────────┘└──┬───┘
  │           │         │      │              │             │
Protocol    Host      Port   Path          Query        Fragment
```

### Question 1.1 (Solved): Parse a URL

Use Python's `urllib.parse` to break down a URL into its components.

In [3]:
# SOLVED EXAMPLE
from urllib.parse import urlparse, parse_qs

url = "https://api.omdbapi.com/?apikey=demo&t=Inception&y=2010#details"

parsed = urlparse(url)

print(f"Scheme (protocol): {parsed.scheme}")
print(f"Host (domain): {parsed.netloc}")
print(f"Path: {parsed.path}")
print(f"Query string: {parsed.query}")
print(f"Fragment: {parsed.fragment}")

# Parse query parameters into a dictionary
params = parse_qs(parsed.query)
print(f"\nParsed parameters: {params}")

Scheme (protocol): https
Host (domain): api.omdbapi.com
Path: /
Query string: apikey=demo&t=Inception&y=2010
Fragment: details

Parsed parameters: {'apikey': ['demo'], 't': ['Inception'], 'y': ['2010']}


### Question 1.2: Parse a Different URL

Parse the following GitHub API URL and extract:
1. The host
2. The path
3. All query parameters as a dictionary

URL: `https://api.github.com/search/repositories?q=machine+learning&sort=stars&order=desc`

In [4]:
# YOUR CODE HERE
url = "https://api.github.com/search/repositories?q=machine+learning&sort=stars&order=desc"

# Parse the URL
parsed_url = urlparse(url)
# Print the host
print(f"Host of the URL: {parsed_url.netloc}")  # What is the difference between netloc and hostname I think both are the same
# Print the path
print(f"path of the URL: {parsed_url.path}")
# Print the query parameters as a dictionary
print(parsed_url.scheme)
print(parsed_url.params)
print(parsed_url.fragment)
print(parsed_url.query)

params = parse_qs(parsed_url.query)
print(f"Params of the URL: {params}")
# print(parsed_url.encode)

Host of the URL: api.github.com
path of the URL: /search/repositories
https


q=machine+learning&sort=stars&order=desc
Params of the URL: {'q': ['machine learning'], 'sort': ['stars'], 'order': ['desc']}


---

## 1.2 HTTP Status Codes

HTTP status codes tell you what happened with your request:

| Range | Category | Common Examples |
|-------|----------|----------------|
| 2xx | Success | 200 OK, 201 Created |
| 3xx | Redirect | 301 Moved, 302 Found |
| 4xx | Client Error | 400 Bad Request, 401 Unauthorized, 404 Not Found |
| 5xx | Server Error | 500 Internal Error, 503 Service Unavailable |

### Question 1.3: Match Status Codes

Match each scenario to the most likely HTTP status code:

1. You requested a movie that doesn't exist in the database
2. You made too many requests and hit the rate limit
3. Your API key is invalid
4. The request was successful and data was returned
5. The server crashed while processing your request

Status codes to choose from: `200`, `401`, `404`, `429`, `500`

In [5]:
# YOUR ANSWERS HERE
answers = {
    "movie_not_found": 404,      # Replace None with the status code
    "rate_limited": 429,
    "invalid_api_key": 401,
    "success": 200,
    "server_crashed": 500
}

print(answers)

{'movie_not_found': 404, 'rate_limited': 429, 'invalid_api_key': 401, 'success': 200, 'server_crashed': 500}


---

# Part 2: Making Requests with `curl`

`curl` is a command-line tool for making HTTP requests. It's essential for quick testing.

## 2.1 Basic curl Commands

You can run shell commands in Jupyter using `!` prefix.

### Question 2.1 (Solved): Your First API Call

Let's call a simple public API that requires no authentication.

In [6]:
# SOLVED EXAMPLE
# JSONPlaceholder is a free fake API for testing
!curl -s "https://jsonplaceholder.typicode.com/posts/1"

{
  "userId": 1,
  "id": 1,
  "title": "sunt aut facere repellat provident occaecati excepturi optio reprehenderit",
  "body": "quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto"
}


### Question 2.2: Pretty Print with jq

The output above is hard to read. Use `jq` to format it nicely.

**Hint**: Pipe the curl output to jq: `curl ... | jq .`

In [7]:
!curl --help

Usage: curl [options...] <url>
 -d, --data <data>           HTTP POST data
 -f, --fail                  Fail fast with no output on HTTP errors
 -h, --help <subject>        Get help for commands
 -o, --output <file>         Write to file instead of stdout
 -O, --remote-name           Write output to file named as remote file
 -i, --show-headers          Show response headers in output
 -s, --silent                Silent mode
 -T, --upload-file <file>    Transfer local FILE to destination
 -u, --user <user:password>  Server user and password
 -A, --user-agent <name>     Send User-Agent <name> to server
 -v, --verbose               Make the operation more talkative
 -V, --version               Show version number and quit

This is not the full help; this menu is split into categories.
Use "--help category" to get an overview of all categories, which are:
auth, connection, curl, deprecated, dns, file, ftp, global, http, imap, ldap, 
output, pop3, post, proxy, scp, sftp, smtp, ssh, telnet,

Exploring different functions

In [8]:
!curl -d "https://jsonplaceholder.typicode.com/posts/1" | jq

'jq' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
!curl -f "https://jsonplaceholder.typicode.com/posts/1" | jq

'jq' is not recognized as an internal or external command,
operable program or batch file.


In [10]:
!curl -i "https://jsonplaceholder.typicode.com/posts/1" | jq

'jq' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
!curl -O  "https://jsonplaceholder.typicode.com/posts/1" | jq

'jq' is not recognized as an internal or external command,
operable program or batch file.


In [12]:
!curl -s "https://jsonplaceholder.typicode.com/posts/1" | jq

'jq' is not recognized as an internal or external command,
operable program or batch file.


In [13]:
# YOUR CODE HERE
# Fetch the same post but format the output with jq
!curl -v "https://jsonplaceholder.typicode.com/posts/1" | jq

'jq' is not recognized as an internal or external command,
operable program or batch file.


### Question 2.3: Extract Specific Fields with jq

Fetch all posts from `https://jsonplaceholder.typicode.com/posts` and extract only the `title` field from each post.

**Hint**: Use `jq '.[].title'` to get the title from each element in the array.

In [14]:
# YOUR CODE HERE
!curl -s "https://jsonplaceholder.typicode.com/posts" | jq

'jq' is not recognized as an internal or external command,
operable program or batch file.


We want to get title from each parts

In [15]:
!curl -s "https://jsonplaceholder.typicode.com/posts" | jq '.[].title'

'jq' is not recognized as an internal or external command,
operable program or batch file.


### Question 2.4: View Response Headers

Use the `-I` flag to fetch only the response headers (no body) from:
`https://api.github.com`

What is the value of the `X-RateLimit-Limit` header?

In [16]:
# YOUR CODE HERE
!curl -i "https://api.github.com"

HTTP/1.1 403 rate limit exceeded
Date: Sat, 10 Jan 2026 08:55:19 GMT
Server: Varnish
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'
Access-Control-Allow-Origin: *
Access-Control-Expose-Headers: ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, X-RateLimit-Used, X-RateLimit-Resource, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, Deprecation, Sunset
Content-Type: application/json; charset=utf-8
Referrer-Policy: origin-when-cross-origin, strict-origin-when-cross-origin
X-GitHub-Media-Type: github.v3; format=json
X-RateLimit-Limit: 60
X-RateLimit-Remaining: 0
X-RateLimit-Reset: 1768037256
X-RateLimit-Resource: core
X-RateLimit-Used: 60
Content-Length: 279
X-GitHub-Request-Id: DC0B:267088:FE50A0:12DE783:696213F7

{"messa

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   279  100   279    0     0    915      0 --:--:-- --:--:-- --:--:--   920


Code to get a specific tag from the header

In [17]:
!curl -I -s https://api.github.com | grep -i x-ratelimit-limit:

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [18]:
!curl -I -s https://api.github.com | grep -i ratelimit

'grep' is not recognized as an internal or external command,
operable program or batch file.


### Question 2.5: Add Custom Headers

Make a request to `https://httpbin.org/headers` with the following custom headers:
- `User-Agent: CS203-Lab/1.0`
- `Accept: application/json`

**Hint**: Use `-H "Header-Name: value"` for each header.

In [19]:
# If I want to do multiple oprations on one line I can do something like this
!curl -H "User-Agent: CS203-Lab/1.0" -H "Accept: application/json" "https://httpbin.org/headers"

{
  "headers": {
    "Accept": "application/json", 
    "Host": "httpbin.org", 
    "User-Agent": "CS203-Lab/1.0", 
    "X-Amzn-Trace-Id": "Root=1-696213f9-3ce7c8b86f4067cd2a71d8ad"
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100   188  100   188    0     0    109      0  0:00:01  0:00:01 --:--:--   109


---

# Part 3: Python `requests` Library

While `curl` is great for testing, we need Python for automation.

## 3.1 Basic GET Requests

### Question 3.1 (Solved): Simple GET Request

Make a GET request and inspect the response object.

In [20]:
# SOLVED EXAMPLE
import requests

response = requests.get("https://jsonplaceholder.typicode.com/posts/2")

print(f"Status Code: {response.status_code}")
print(f"Content-Type: {response.headers['Content-Type']}")
print(f"Response OK: {response.ok}")
print(f"\nJSON Data:")
print(response.json())

Status Code: 200
Content-Type: application/json; charset=utf-8
Response OK: True

JSON Data:
{'userId': 1, 'id': 2, 'title': 'qui est esse', 'body': 'est rerum tempore vitae\nsequi sint nihil reprehenderit dolor beatae ea dolores neque\nfugiat blanditiis voluptate porro vel nihil molestiae ut reiciendis\nqui aperiam non debitis possimus qui neque nisi nulla'}


### Question 3.2: Fetch Multiple Posts

Fetch posts from `https://jsonplaceholder.typicode.com/posts` and:
1. Print the total number of posts
2. Print the titles of the first 5 posts

In [21]:
# YOUR CODE HERE
response = requests.get("https://jsonplaceholder.typicode.com/posts")
print(f'Total no of posts: {len(response.json())}')
print(f'First five titles are here:')
for i in range(5):
  print(f"Title {i+1}: {response.json()[i]['title']}")

Total no of posts: 100
First five titles are here:
Title 1: sunt aut facere repellat provident occaecati excepturi optio reprehenderit
Title 2: qui est esse
Title 3: ea molestias quasi exercitationem repellat qui ipsa sit aut
Title 4: eum et est occaecati
Title 5: nesciunt quas odio


### Question 3.3 (Solved): Using Query Parameters

The proper way to add query parameters is using the `params` argument.

In [22]:
# SOLVED EXAMPLE
import requests

# Bad way (manual string building)
# url = "https://jsonplaceholder.typicode.com/posts?userId=1"

# Good way (using params)
response = requests.get(
    "https://jsonplaceholder.typicode.com/posts",
    params={"userId": 10}
)

posts = response.json()
print(f"User 1 has {len(posts)} posts")
print(f"\nActual URL used: {response.url}")

User 1 has 10 posts

Actual URL used: https://jsonplaceholder.typicode.com/posts?userId=10


### Question 3.4: Filter Posts by User

Fetch all posts by user 5 and user 7. Compare how many posts each user has.

**Hint**: Make two separate requests with different `userId` values.

In [23]:
# YOUR CODE HERE
response_user_5 = requests.get(
    "https://jsonplaceholder.typicode.com/posts",
    params={"userId": 5}
)
posts_5 = response_user_5.json()
response_user_7 = requests.get(
    "https://jsonplaceholder.typicode.com/posts",
    params={"userId": 7}
)
posts_7 = response_user_7.json()
print(f"Posts by user 5 is: {len(posts_5)}, and by user 7 is: {len(posts_7)}")


Posts by user 5 is: 10, and by user 7 is: 10


---

## 3.2 Working with Real APIs

Let's work with some real-world APIs.

### Question 3.5 (Solved): GitHub API - Public Repositories

The GitHub API is free to use (with rate limits) and doesn't require authentication for public data.

In [ ]:
token = "YOUR_TOKEN_HERE"
print("Token exists:", token is not None)

headers = {
    "Authorization": f"token {token}",
    "Accept": "application/vnd.github.v3+json"
}

r = requests.get("https://api.github.com/user", headers=headers)
print(r.status_code)
print(r.json())


Token exists: True
200
{'login': 'pro-laxmi', 'id': 180728857, 'node_id': 'U_kgDOCsW0GQ', 'avatar_url': 'https://avatars.githubusercontent.com/u/180728857?v=4', 'gravatar_id': '', 'url': 'https://api.github.com/users/pro-laxmi', 'html_url': 'https://github.com/pro-laxmi', 'followers_url': 'https://api.github.com/users/pro-laxmi/followers', 'following_url': 'https://api.github.com/users/pro-laxmi/following{/other_user}', 'gists_url': 'https://api.github.com/users/pro-laxmi/gists{/gist_id}', 'starred_url': 'https://api.github.com/users/pro-laxmi/starred{/owner}{/repo}', 'subscriptions_url': 'https://api.github.com/users/pro-laxmi/subscriptions', 'organizations_url': 'https://api.github.com/users/pro-laxmi/orgs', 'repos_url': 'https://api.github.com/users/pro-laxmi/repos', 'events_url': 'https://api.github.com/users/pro-laxmi/events{/privacy}', 'received_events_url': 'https://api.github.com/users/pro-laxmi/received_events', 'type': 'User', 'user_view_type': 'public', 'site_admin': False, 

In [41]:
# SOLVED EXAMPLE
import requests

# Fetch information about a popular repository
headers = {
    "Authorization": f"token {token}",
    "Accept": "application/vnd.github.v3+json"
}
response = requests.get(
    "https://api.github.com/repos/pandas-dev/pandas",
    headers=headers
)

if response.ok:
    repo = response.json()
    print(f"Repository: {repo['full_name']}")
    print(f"Description: {repo['description']}")
    print(f"Stars: {repo['stargazers_count']:,}")
    print(f"Forks: {repo['forks_count']:,}")
    print(f"Language: {repo['language']}")
else:
    print(f"Error: {response.status_code}")

Repository: pandas-dev/pandas
Description: Flexible and powerful data analysis / manipulation library for Python, providing labeled data structures similar to R data.frame objects, statistical functions, and much more
Stars: 47,520
Forks: 19,492
Language: Python


In [38]:
!curl -I -s https://api.github.com/repos/pandas-dev/pandas

HTTP/1.1 403 rate limit exceeded
Date: Sat, 10 Jan 2026 09:08:00 GMT
Server: Varnish
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'
Access-Control-Allow-Origin: *
Access-Control-Expose-Headers: ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, X-RateLimit-Used, X-RateLimit-Resource, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, Deprecation, Sunset
Content-Type: application/json; charset=utf-8
Referrer-Policy: origin-when-cross-origin, strict-origin-when-cross-origin
X-GitHub-Media-Type: github.v3; format=json
X-RateLimit-Limit: 60
X-RateLimit-Remaining: 0
X-RateLimit-Reset: 1768037256
X-RateLimit-Resource: core
X-RateLimit-Used: 60
Content-Length: 279
X-GitHub-Request-Id: C88B:15FCE6:FC307E:12C2206:696216F0



### Question 3.6: Compare Popular ML Libraries

Fetch information about these ML-related repositories and create a comparison table:
- `scikit-learn/scikit-learn`
- `pytorch/pytorch`
- `tensorflow/tensorflow`

Show: name, stars, forks, and primary language.

**Hint**: Loop through the repos and collect data into a list of dictionaries, then create a DataFrame.

In [42]:
# YOUR CODE HERE
repos = [
    "scikit-learn/scikit-learn",
    "pytorch/pytorch",
    "tensorflow/tensorflow"
]

all_repos_data = []
# Fetch data for each repo
for repo in repos:
    response = requests.get(
        f"https://api.github.com/repos/{repo}",
        headers=headers
    )
    if response.ok:
        data = response.json()
        all_repos_data.append({'repo': data['full_name'], 'stars': data['stargazers_count'], 'forks': data['forks_count'], 'language': data['language']})
        print(f"Repository: {data['full_name']}")
        print(f"Stars: {data['stargazers_count']:,}")
        print(f"Forks: {data['forks_count']:,}")
        print(f"Language: {data['language']}")
        print("-" * 40)
    else:
        print(f"Error fetching {repo}: {response.status_code}")
# Create a DataFrame
repo_data = pd.DataFrame(all_repos_data)

# Display the comparison
repo_data

Repository: scikit-learn/scikit-learn
Stars: 64,573
Forks: 26,585
Language: Python
----------------------------------------
Repository: pytorch/pytorch
Stars: 96,481
Forks: 26,473
Language: Python
----------------------------------------
Repository: tensorflow/tensorflow
Stars: 193,273
Forks: 75,148
Language: C++
----------------------------------------


,repo,stars,forks,language
0,scikit-learn/scikit-learn,64573,26585,Python
1,pytorch/pytorch,96481,26473,Python
2,tensorflow/tensorflow,193273,75148,C++


### Question 3.7: Search GitHub Repositories

Use the GitHub search API to find the top 10 most starred repositories with "machine learning" in their description.

API endpoint: `https://api.github.com/search/repositories`

Parameters:
- `q`: search query (e.g., "machine learning")
- `sort`: "stars"
- `order`: "desc"
- `per_page`: 10

Print the name and star count of each repository.

In [44]:
# YOUR CODE HERE
url = "https://api.github.com/search/repositories"
params = {
    'q' : "machine learning",
    'sort' : "stars",
    'order' : "desc",
    'per_page' : 10
}

response = requests.get(url, headers=headers, params=params)
data = response.json()
all_repos_data = []
for item in data['items']:
    print(f'Name of the repo: {item['name']} and the star count is: {item['stargazers_count']}')

Name of the repo: tensorflow and the star count is: 193273
Name of the repo: transformers and the star count is: 154836
Name of the repo: ML-For-Beginners and the star count is: 82967
Name of the repo: funNLP and the star count is: 78348
Name of the repo: awesome-machine-learning and the star count is: 71266
Name of the repo: scikit-learn and the star count is: 64573
Name of the repo: gradio and the star count is: 41247
Name of the repo: C-Plus-Plus and the star count is: 33658
Name of the repo: netron and the star count is: 32157
Name of the repo: 500-AI-Machine-learning-Deep-learning-Computer-vision-NLP-Projects-with-code and the star count is: 30752


---

## 3.3 Error Handling

Real-world APIs fail. We need to handle errors gracefully.

### Question 3.8 (Solved): Handling HTTP Errors

In [45]:
# SOLVED EXAMPLE
import requests

def fetch_with_error_handling(url):
    """Fetch URL with proper error handling."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raises exception for 4xx/5xx
        return response.json()
    except requests.exceptions.Timeout:
        print(f"Timeout: Request took too long")
    except requests.exceptions.HTTPError as e:
        print(f"HTTP Error: {e.response.status_code}")
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    return None

# Test with valid URL
print("Valid URL:")
data = fetch_with_error_handling("https://jsonplaceholder.typicode.com/posts/1")
if data:
    print(f"  Got post: {data['title'][:50]}...")

# Test with invalid URL (404)
print("\nInvalid URL (404):")
fetch_with_error_handling("https://jsonplaceholder.typicode.com/posts/99999")

Valid URL:
  Got post: sunt aut facere repellat provident occaecati excep...

Invalid URL (404):
HTTP Error: 404


### Question 3.9: Robust Fetcher Function

Write a function `safe_fetch(url, max_retries=3)` that:

1. Attempts to fetch the URL
2. If it fails with a 5xx error, retries up to `max_retries` times
3. Waits 1 second between retries
4. Returns the JSON data if successful, None otherwise

Test it with `https://httpbin.org/status/500` (always returns 500) and `https://jsonplaceholder.typicode.com/posts/1` (always works).

In [53]:
# YOUR CODE HERE
import time

def safe_fetch(url, max_retries=3):
    """Fetch URL with retry logic for server errors."""
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raises exception for 4xx/5xx
        return response.json()
    except requests.exceptions.RequestException as e:
        if (500<=e.response.status_code<600):
            print(f'Trying my ass off.....')
            for i in range(max_retries):
                try:
                    response = requests.get(url, timeout=10)
                    response.raise_for_status()  # Raises exception for 4xx/5xx
                    return response.json()
                except requests.exceptions.RequestException as e:
                    if (500<=e.response.status_code<600):
                        print(f'Filed after {i+1} tries')
            
    return "Fuck I can't do this anymore 😭😭"
# Test your function
print("Testing with working URL:")
result = safe_fetch("https://jsonplaceholder.typicode.com/posts/1")
print(f"Result: {result}")

print("\nTesting with failing URL (500):")
result = safe_fetch("https://httpbin.org/status/500")
print(f"Result: {result}")

Testing with working URL:
Result: {'userId': 1, 'id': 1, 'title': 'sunt aut facere repellat provident occaecati excepturi optio reprehenderit', 'body': 'quia et suscipit\nsuscipit recusandae consequuntur expedita et cum\nreprehenderit molestiae ut ut quas totam\nnostrum rerum est autem sunt rem eveniet architecto'}

Testing with failing URL (500):
Trying my ass off.....
Filed after 1 tries
Filed after 2 tries
Filed after 3 tries
Result: Fuck I can't do this anymore 😭😭


---

# Part 4: The OMDb Movie API

Now let's work with the OMDb API - our main data source for the Netflix project.

**Note**: You need an API key from https://www.omdbapi.com/apikey.aspx (free tier available).

For this lab, we'll use a demo key that has limited functionality.

In [1]:
# Set your API key here
# Get a free key from: https://www.omdbapi.com/apikey.aspx
OMDB_API_KEY = "1baeaa9" # Replace with your actual key

# For demo purposes, you can try with key "demo" but it's very limited
# OMDB_API_KEY = "demo"

### Question 4.1 (Solved): Fetch a Single Movie

In [2]:
# SOLVED EXAMPLE
import requests

def fetch_movie(title, year=None, api_key=OMDB_API_KEY):
    """Fetch movie data from OMDb API."""
    params = {
        "apikey": api_key,
        "t": title,  # Search by title
        "type": "movie"
    }
    if year:
        params["y"] = year

    response = requests.get("https://www.omdbapi.com/", params=params)

    if response.ok:
        data = response.json()
        if data.get("Response") == "True":
            return data
        else:
            print(f"Movie not found: {data.get('Error')}")
    return None

# Fetch Inception
movie = fetch_movie("Inception", 2010)
if movie:
    print(f"Title: {movie['Title']}")
    print(f"Year: {movie['Year']}")
    print(f"Director: {movie['Director']}")
    print(f"IMDB Rating: {movie['imdbRating']}")
    print(f"Genre: {movie['Genre']}")
movie

Title: Inception
Year: 2010
Director: Christopher Nolan
IMDB Rating: 8.8
Genre: Action, Adventure, Sci-Fi


{'Title': 'Inception',
 'Year': '2010',
 'Rated': 'PG-13',
 'Released': '16 Jul 2010',
 'Runtime': '148 min',
 'Genre': 'Action, Adventure, Sci-Fi',
 'Director': 'Christopher Nolan',
 'Writer': 'Christopher Nolan',
 'Actors': 'Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page',
 'Plot': 'A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a CEO, but his tragic past may doom the project and his team to disaster.',
 'Language': 'English, Japanese, French',
 'Country': 'United States, United Kingdom',
 'Awards': 'Won 4 Oscars. 159 wins & 220 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMjAxMzY3NjcxNF5BMl5BanBnXkFtZTcwNTI5OTM0Mw@@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.8/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '87%'},
  {'Source': 'Metacritic', 'Value': '74/100'}],
 'Metascore': '74',
 'imdbRating': '8.8',
 'imdbVotes': '2,

### Question 4.2: Explore the Response

Fetch data for "The Dark Knight" and print ALL available fields in the response.

Which fields might be useful for predicting movie success?

In [80]:
# YOUR CODE HERE
dark_knight = fetch_movie("The Dark Knight")
dark_knight

{'Title': 'The Dark Knight',
 'Year': '2008',
 'Rated': 'PG-13',
 'Released': '18 Jul 2008',
 'Runtime': '152 min',
 'Genre': 'Action, Crime, Drama',
 'Director': 'Christopher Nolan',
 'Writer': 'Jonathan Nolan, Christopher Nolan, David S. Goyer',
 'Actors': 'Christian Bale, Heath Ledger, Aaron Eckhart',
 'Plot': 'When a menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman, James Gordon and Harvey Dent must work together to put an end to the madness.',
 'Language': 'English, Mandarin',
 'Country': 'United States, United Kingdom',
 'Awards': 'Won 2 Oscars. 163 wins & 165 nominations total',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTMxNTMwODM0NF5BMl5BanBnXkFtZTcwODAyMTk2Mw@@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '9.1/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '94%'},
  {'Source': 'Metacritic', 'Value': '85/100'}],
 'Metascore': '85',
 'imdbRating': '9.1',
 'imdbVotes': '3,115,102',
 'imdbID': 'tt0468569',
 

### Question 4.3: Fetch Multiple Movies

Create a function `fetch_movies(titles)` that:
1. Takes a list of movie titles
2. Fetches data for each movie
3. Returns a list of movie dictionaries (only successful fetches)
4. Adds a 0.5 second delay between requests (to respect rate limits)

Test it with: `["Inception", "The Matrix", "Interstellar", "NonExistentMovie123"]`

In [87]:
# YOUR CODE HERE
def fetch_movies(titles, api_key = OMDB_API_KEY):
    """Fetch multiple movies from OMDb API."""
    success_movie_fetch = []
    for title in titles:
        params = {
            "apikey": api_key,
            "t": title,  # Search by title
            "type": "movie"
        }

        response = requests.get("https://www.omdbapi.com/", params=params)

        if response.ok:
            data = response.json()
            if data.get("Response") == "True":
                success_movie_fetch.append(data)
            else:
                print(f"Movie not found: {data.get('Error')}")
        time.sleep(0.5)
    return success_movie_fetch

# Test
test_titles = ["Inception", "The Matrix", "Interstellar", "NonExistentMovie123"]
movies = fetch_movies(test_titles)
print(f"Successfully fetched {len(movies)} out of {len(test_titles)} movies")

Movie not found: Movie not found!
Successfully fetched 3 out of 4 movies


In [90]:
movies

[{'Title': 'Inception',
  'Year': '2010',
  'Rated': 'PG-13',
  'Released': '16 Jul 2010',
  'Runtime': '148 min',
  'Genre': 'Action, Adventure, Sci-Fi',
  'Director': 'Christopher Nolan',
  'Writer': 'Christopher Nolan',
  'Actors': 'Leonardo DiCaprio, Joseph Gordon-Levitt, Elliot Page',
  'Plot': 'A thief who steals corporate secrets through the use of dream-sharing technology is given the inverse task of planting an idea into the mind of a CEO, but his tragic past may doom the project and his team to disaster.',
  'Language': 'English, Japanese, French',
  'Country': 'United States, United Kingdom',
  'Awards': 'Won 4 Oscars. 159 wins & 220 nominations total',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BMjAxMzY3NjcxNF5BMl5BanBnXkFtZTcwNTI5OTM0Mw@@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '8.8/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '87%'},
   {'Source': 'Metacritic', 'Value': '74/100'}],
  'Metascore': '74',
  'imdbRating': '8.8'

### Question 4.4: Create a Movie DataFrame

Using the movies you fetched, create a pandas DataFrame with these columns:
- title
- year (as integer)
- genre
- director
- imdb_rating (as float)
- imdb_votes (as integer, remove commas)
- runtime_minutes (as integer, extract from "148 min")
- box_office (keep as string for now)

**Hint**: You'll need to clean the data types.

In [101]:
# YOUR CODE HERE
import warnings
warnings.filterwarnings("ignore")

df = pd.DataFrame(movies)
df_required = df[['Title', 'Year', 'Director', 'imdbRating', 'Genre', 'imdbVotes', 'Runtime', 'BoxOffice']]
df_required['Year'] = pd.to_numeric(df_required['Year'], errors='coerce').astype(int)
df_required['imdbRating'] = pd.to_numeric(df_required['imdbRating'], errors='coerce').astype(float)
df_required['imdbVotes'] = df_required['imdbVotes'].str.replace(',', '').astype(int)
df_required['Runtime'] = df_required['Runtime'].str.replace(' min', '').astype(int)
df_required

,Title,Year,Director,imdbRating,Genre,imdbVotes,Runtime,BoxOffice
0,Inception,2010,Christopher Nolan,8.8,"Action, Adventure, Sci-Fi",2767518,148,"$292,587,330"
1,The Matrix,1999,"Lana Wachowski, Lilly Wachowski",8.7,"Action, Sci-Fi",2217731,136,"$177,559,005"
2,Interstellar,2014,Christopher Nolan,8.7,"Adventure, Drama, Sci-Fi",2454660,169,"$203,227,580"


### Question 4.5: Search Movies by Title

OMDb also has a search endpoint that returns multiple results.

Use the `s` parameter instead of `t` to search for movies containing "Star Wars".

API endpoint: `https://www.omdbapi.com/?apikey=YOUR_KEY&s=Star Wars&type=movie`

Print the title and year of each result.

In [109]:
response = requests.get("https://www.omdbapi.com/?apikey=1baeaa9&s=Star Wars&type=movie")
for search in response.json()['Search']:
    print(f"The title of the movie: {search['Title']}, and the year is: {search['Year']}")

The title of the movie: Star Wars: Episode IV - A New Hope, and the year is: 1977
The title of the movie: Star Wars: Episode V - The Empire Strikes Back, and the year is: 1980
The title of the movie: Star Wars: Episode VI - Return of the Jedi, and the year is: 1983
The title of the movie: Star Wars: Episode VII - The Force Awakens, and the year is: 2015
The title of the movie: Star Wars: Episode I - The Phantom Menace, and the year is: 1999
The title of the movie: Star Wars: Episode III - Revenge of the Sith, and the year is: 2005
The title of the movie: Star Wars: Episode II - Attack of the Clones, and the year is: 2002
The title of the movie: Rogue One: A Star Wars Story, and the year is: 2016
The title of the movie: Star Wars: Episode VIII - The Last Jedi, and the year is: 2017
The title of the movie: Star Wars: Episode IX - The Rise of Skywalker, and the year is: 2019


In [106]:
response.json()

{'Search': [{'Title': 'Star Wars: Episode IV - A New Hope',
   'Year': '1977',
   'imdbID': 'tt0076759',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BOGUwMDk0Y2MtNjBlNi00NmRiLTk2MWYtMGMyMDlhYmI4ZDBjXkEyXkFqcGc@._V1_SX300.jpg'},
  {'Title': 'Star Wars: Episode V - The Empire Strikes Back',
   'Year': '1980',
   'imdbID': 'tt0080684',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BMTkxNGFlNDktZmJkNC00MDdhLTg0MTEtZjZiYWI3MGE5NWIwXkEyXkFqcGc@._V1_SX300.jpg'},
  {'Title': 'Star Wars: Episode VI - Return of the Jedi',
   'Year': '1983',
   'imdbID': 'tt0086190',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BNWEwOTI0MmUtMGNmNy00ODViLTlkZDQtZTg1YmQ3MDgyNTUzXkEyXkFqcGc@._V1_SX300.jpg'},
  {'Title': 'Star Wars: Episode VII - The Force Awakens',
   'Year': '2015',
   'imdbID': 'tt2488496',
   'Type': 'movie',
   'Poster': 'https://m.media-amazon.com/images/M/MV5BOTAzODEzNDAzMl5BMl5BanBnXkFtZTgwMDU1MTgzNzE@._V1_SX3

### Question 4.6: Handle Pagination

The OMDb search API returns 10 results per page and includes a `totalResults` field.

Write a function `search_all_movies(query)` that:
1. Searches for movies matching the query
2. Fetches ALL pages of results (use the `page` parameter)
3. Returns a list of all movies found

**Hint**: `totalResults` tells you how many movies exist. Divide by 10 to get the number of pages.

Test with a query that has many results like "Batman".

In [115]:
# YOUR CODE HERE
def search_all_movies(query, api_key=OMDB_API_KEY):
    """Search OMDb and return ALL matching movies across all pages."""
    all_batman_movies = []
    params = {
        "apikey": api_key,
        "s": query,  # Search by title
        "type": "movie",
    }
    response = requests.get("https://www.omdbapi.com/", params=params)
    for i in range(int(response.json()['totalResults'])//10+1):
        params = {
            "apikey": api_key,
            "s": query,  # Search by title
            "type": "movie",
            "page": i+1
        }
        response = requests.get("https://www.omdbapi.com/", params=params)
        for search in response.json()['Search']:
            all_batman_movies.append(search['Title'])

    return all_batman_movies
# Test
all_batman = search_all_movies("Batman")
print(f"Found {len(all_batman)} Batman movies")

Found 514 Batman movies


---

# Part 5: Web Scraping with BeautifulSoup

When APIs don't exist or don't have what we need, we scrape.

## 5.1 HTML Basics

### Question 5.1 (Solved): Parse HTML

In [2]:
# SOLVED EXAMPLE
from bs4 import BeautifulSoup

html = """
<html>
<body>
    <div class="movie" id="movie-1">
        <h2 class="title">Inception</h2>
        <span class="year">2010</span>
        <span class="rating">8.8</span>
        <a href="/movies/inception">More Info</a>
    </div>
    <div class="movie" id="movie-2">
        <h2 class="title">The Matrix</h2>
        <span class="year">1999</span>
        <span class="rating">8.7</span>
        <a href="/movies/matrix">More Info</a>
    </div>
</body>
</html>
"""

soup = BeautifulSoup(html, 'html.parser')

# Find all movie divs
movies = soup.find_all('div', class_='movie')
print(f"Found {len(movies)} movies\n")

# Extract data from each
for movie in movies:
    title = movie.find('h2', class_='title').text
    year = movie.find('span', class_='year').text
    rating = movie.find('span', class_='rating').text
    link = movie.find('a')['href']

    print(f"{title} ({year}) - Rating: {rating} - Link: {link}")

Found 2 movies

Inception (2010) - Rating: 8.8 - Link: /movies/inception
The Matrix (1999) - Rating: 8.7 - Link: /movies/matrix


### Question 5.2: CSS Selectors

Rewrite the above extraction using CSS selectors (`.select()` and `.select_one()`) instead of `.find()` and `.find_all()`.

**Hint**:
- `.movie` selects elements with class "movie"
- `.movie .title` selects elements with class "title" inside class "movie"

In [17]:
# YOUR CODE HERE
# Use the same 'soup' from above
soup = BeautifulSoup(html, 'html.parser')

# Find all movie divs
movies = soup.find_all('div', class_='movie')
print(f"Found {len(movies)} movies\n")

# Extract data from each
for movie in movies:
    title = movie.find('h2', class_='title').text
    year = movie.find('span', class_='year').text
    rating = movie.find('span', class_='rating').text
    link = movie.find('a')['href']

    print(f"{title} ({year}) - Rating: {rating} - Link: {link}")

# Extract using CSS selectors
keywords = soup.select('div.movie')
print(f'Found {len(keywords)} titles using CSS selectors:\n')
for keyword in keywords:
    title = keyword.find('h2', class_='title').text
    year = keyword.find('span', class_='year').text
    rating = keyword.find('span', class_='rating').text
    link = keyword.find('a')['href']
    print(f"{title} ({year}) - Rating: {rating} - Link: {link}")


Found 2 movies

Inception (2010) - Rating: 8.8 - Link: /movies/inception
The Matrix (1999) - Rating: 8.7 - Link: /movies/matrix
Found 2 titles using CSS selectors:

Inception (2010) - Rating: 8.8 - Link: /movies/inception
The Matrix (1999) - Rating: 8.7 - Link: /movies/matrix


These two lines of code essentially do the same search over movies

In [18]:
key = soup.select('div.movie')
mov = soup.find_all('div', class_='movie')
key, mov

([<div class="movie" id="movie-1">
  <h2 class="title">Inception</h2>
  <span class="year">2010</span>
  <span class="rating">8.8</span>
  <a href="/movies/inception">More Info</a>
  </div>,
  <div class="movie" id="movie-2">
  <h2 class="title">The Matrix</h2>
  <span class="year">1999</span>
  <span class="rating">8.7</span>
  <a href="/movies/matrix">More Info</a>
  </div>],
 [<div class="movie" id="movie-1">
  <h2 class="title">Inception</h2>
  <span class="year">2010</span>
  <span class="rating">8.8</span>
  <a href="/movies/inception">More Info</a>
  </div>,
  <div class="movie" id="movie-2">
  <h2 class="title">The Matrix</h2>
  <span class="year">1999</span>
  <span class="rating">8.7</span>
  <a href="/movies/matrix">More Info</a>
  </div>])

### Question 5.3: Scrape a Real Website

Let's scrape the example website `http://quotes.toscrape.com/` which is designed for scraping practice.

Extract all quotes from the first page, including:
- The quote text
- The author name
- The tags

Return the results as a list of dictionaries.

In [ ]:
# YOUR CODE HERE
import requests
from bs4 import BeautifulSoup

# Fetch the page
url = "http://quotes.toscrape.com/"
try:
    response = requests.get(url)
    response.raise_for_status() # Raise an exception for bad status codes
    print(response.status_code)

    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

# Extract quotes
quotes = soup.select('div.quote > span.text')

# Print results
print(f'Found {len(quotes)} quotes:\n')
for quote in quotes:
    print(quote.text)


200
Found 10 quotes:

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“It is our choices, Harry, that show what we truly are, far more than our abilities.”
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
“Try not to become a man of success. Rather become a man of value.”
“It is better to be hated for what you are than to be loved for what you are not.”
“I have not failed. I've just found 10,000 ways that won't work.”
“A woman is like a tea bag; you never know how strong it is until it's in hot water.”
“A day without sunshine is like, you know, night.”


### Question 5.4: Handle Pagination in Scraping

The quotes website has multiple pages. Scrape the first 3 pages and collect all quotes.

Pages follow the pattern:
- Page 1: `http://quotes.toscrape.com/page/1/`
- Page 2: `http://quotes.toscrape.com/page/2/`
- etc.

**Remember**: Add a delay between requests to be polite!

In [37]:
import time

url = "http://quotes.toscrape.com/"
for i in range(1, 4):
    params = {'page': i}
    try:
        response = requests.get(url, params=params)
        response.raise_for_status() # Raise an exception for bad status codes
        print(response.status_code)

        html_content = response.text
        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract quotes
        quotes = soup.select('div.quote > span.text')

        # Print results
        print(f'Found {len(quotes)} quotes in page {i}:\n')
        for quote in quotes:
            print(quote.text)

        time.sleep(1)  # Be polite and avoid overwhelming the server

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

200
Found 10 quotes in page 1:

“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“It is our choices, Harry, that show what we truly are, far more than our abilities.”
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
“The person, be it gentleman or lady, who has not pleasure in a good novel, must be intolerably stupid.”
“Imperfection is beauty, madness is genius and it's better to be absolutely ridiculous than absolutely boring.”
“Try not to become a man of success. Rather become a man of value.”
“It is better to be hated for what you are than to be loved for what you are not.”
“I have not failed. I've just found 10,000 ways that won't work.”
“A woman is like a tea bag; you never know how strong it is until it's in hot water.”
“A day without sunshine is like, you know, night.”
200
Found 10 quotes in page 2:

“The world as we have created i

### Question 5.5: Extract Table Data

Scrape the table from `https://www.w3schools.com/html/html_tables.asp`.

The table contains company data. Extract all rows and create a pandas DataFrame.

**Hint**: Look for `<table>`, `<tr>` (table row), `<th>` (header), and `<td>` (data cell) elements.

In [2]:
# YOUR CODE HERE
# Hint: pandas has a read_html() function that can do this automatically!
# But try doing it manually first to understand the process.
url = "https://www.w3schools.com/html/html_tables.asp"
response = requests.get(url)

html_content = response.text
soup = BeautifulSoup(html_content, 'html.parser')
table = soup.find('table')

for tr in table.find_all('tr'):
    for th in tr.find_all('th'):
        print(th.text, end=" ")
    
    for td in tr.find_all('td'):
        print(td.text, end=' ')
    print()

Company Contact Country 
Alfreds Futterkiste Maria Anders Germany 
Centro comercial Moctezuma Francisco Chang Mexico 
Ernst Handel Roland Mendel Austria 
Island Trading Helen Bennett UK 
Laughing Bacchus Winecellars Yoshi Tannamuri Canada 
Magazzini Alimentari Riuniti Giovanni Rovelli Italy 


In [3]:
df = pd.read_html(str(response.text))[0]
df

/tmp/ipykernel_78659/2525532223.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(response.text))[0]


,Company,Contact,Country
0,Alfreds Futterkiste,Maria Anders,Germany
1,Centro comercial Moctezuma,Francisco Chang,Mexico
2,Ernst Handel,Roland Mendel,Austria
3,Island Trading,Helen Bennett,UK
4,Laughing Bacchus Winecellars,Yoshi Tannamuri,Canada
5,Magazzini Alimentari Riuniti,Giovanni Rovelli,Italy


---

# Part 6: Building the Movie Data Pipeline

Now let's put everything together to build a complete data collection pipeline for our Netflix project.

## 6.1 The Complete Pipeline

### Question 6.1 (Solved): Movie Data Collector Class

In [6]:
# SOLVED EXAMPLE
import requests
import pandas as pd
import time
from typing import List, Dict, Optional

class MovieDataCollector:
    """Collect movie data from OMDb API."""

    def __init__(self, api_key: str):
        self.api_key = api_key
        self.base_url = "http://www.omdbapi.com/"
        self.delay = 0.5  # Seconds between requests

    def fetch_movie(self, title: str, year: Optional[int] = None) -> Optional[Dict]:
        """Fetch a single movie by title."""
        params = {
            "apikey": self.api_key,
            "t": title,
            "type": "movie"
        }
        if year:
            params["y"] = year

        try:
            response = requests.get(self.base_url, params=params, timeout=10)
            response.raise_for_status()
            data = response.json()

            if data.get("Response") == "True":
                return data
        except Exception as e:
            print(f"Error fetching {title}: {e}")

        return None

    def fetch_movies(self, titles: List[str]) -> List[Dict]:
        """Fetch multiple movies."""
        movies = []

        for i, title in enumerate(titles):
            print(f"Fetching {i+1}/{len(titles)}: {title}")
            movie = self.fetch_movie(title)

            if movie:
                movies.append(movie)

            time.sleep(self.delay)

        return movies

    def to_dataframe(self, movies: List[Dict]) -> pd.DataFrame:
        """Convert movie data to cleaned DataFrame."""
        if not movies:
            return pd.DataFrame()

        # Extract relevant fields
        rows = []
        for m in movies:
            rows.append({
                "title": m.get("Title"),
                "year": m.get("Year"),
                "genre": m.get("Genre"),
                "director": m.get("Director"),
                "actors": m.get("Actors"),
                "imdb_rating": m.get("imdbRating"),
                "imdb_votes": m.get("imdbVotes"),
                "runtime": m.get("Runtime"),
                "box_office": m.get("BoxOffice"),
                "imdb_id": m.get("imdbID")
            })

        df = pd.DataFrame(rows)

        # Clean data types
        df["year"] = pd.to_numeric(df["year"], errors="coerce").astype("Int64")
        df["imdb_rating"] = pd.to_numeric(df["imdb_rating"], errors="coerce")
        df["imdb_votes"] = df["imdb_votes"].str.replace(",", "").pipe(pd.to_numeric, errors="coerce").astype("Int64")
        # Fix: str.extract returns a DataFrame, we need column 0 to get a Series
        df["runtime_min"] = df["runtime"].str.extract(r"(\d+)")[0].pipe(pd.to_numeric, errors="coerce").astype("Int64")

        return df

# Usage example
collector = MovieDataCollector(OMDB_API_KEY)
movies = collector.fetch_movies(["Inception", "The Matrix"])
df = collector.to_dataframe(movies)
print(df)

Fetching 1/2: Inception
Fetching 2/2: The Matrix
        title  year                      genre  \
0   Inception  2010  Action, Adventure, Sci-Fi   
1  The Matrix  1999             Action, Sci-Fi   

                          director  \
0                Christopher Nolan   
1  Lana Wachowski, Lilly Wachowski   

                                              actors  imdb_rating  imdb_votes  \
0  Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...          8.8     2767518   
1  Keanu Reeves, Laurence Fishburne, Carrie-Anne ...          8.7     2217731   

   runtime    box_office    imdb_id  runtime_min  
0  148 min  $292,587,330  tt1375666          148  
1  136 min  $177,559,005  tt0133093          136  


### Question 6.2: Add Search Functionality

Extend the `MovieDataCollector` class to add a `search_movies(query, max_results=50)` method that:
1. Searches for movies matching the query
2. Handles pagination to get up to `max_results` movies
3. For each search result, fetches the full movie details
4. Returns the detailed movie data

**Hint**: Search results only contain basic info (title, year, poster, imdbID). You need to use the imdbID to fetch full details.

In [ ]:
# YOUR CODE HERE
# Extend the MovieDataCollector class or add a method


### Question 6.3: Build a Genre-Based Dataset

Use your collector to build a dataset of popular movies from different genres:

1. Search for 10 movies each for: "action", "comedy", "drama", "horror", "sci-fi"
2. Combine all results into a single DataFrame
3. Remove any duplicates (some movies might appear in multiple searches)
4. Save to CSV

**Note**: This might take a while due to rate limiting. Start with fewer movies for testing.

In [ ]:
# YOUR CODE HERE


### Question 6.4: Data Quality Analysis

Using the dataset you created:

1. How many movies have missing IMDB ratings?
2. How many movies have missing box office data?
3. What's the distribution of ratings? (min, max, mean, median)
4. Which directors appear most frequently?
5. What's the average runtime by genre?

These quality checks will be important for Week 2 (Data Validation)!

In [ ]:
# YOUR CODE HERE


---

# Part 7: Challenge Problems

These are optional advanced exercises for those who finish early.

### Challenge 7.1: Rate Limit Handler

Create a `RateLimiter` class that:
1. Tracks how many requests have been made
2. Automatically adds delays to stay under a rate limit
3. Handles 429 (Too Many Requests) responses by waiting and retrying

```python
limiter = RateLimiter(requests_per_minute=30)
response = limiter.get("https://api.example.com/data")
```

In [ ]:
# YOUR CODE HERE


### Challenge 7.2: Async Movie Collector

The synchronous approach is slow because we wait for each request to complete.

Create an async version using `aiohttp` that can fetch multiple movies concurrently (while still respecting rate limits).

Compare the time to fetch 20 movies with sync vs async approach.

In [ ]:
# YOUR CODE HERE
# Hint: You'll need to install aiohttp: pip install aiohttp
# And use asyncio to run the async code


### Challenge 7.3: Multi-Source Data Fusion

Create a data collection pipeline that:
1. Fetches basic movie data from OMDb
2. Enriches it with additional data from another source (e.g., Wikipedia API for plot summaries)
3. Merges the data based on movie title/year
4. Handles cases where data is missing from one source

Wikipedia API example:
```
https://en.wikipedia.org/api/rest_v1/page/summary/Inception_(film)
```

In [ ]:
# YOUR CODE HERE


---

# Summary

In this lab, you learned:

1. **HTTP Fundamentals**: URLs, status codes, headers
2. **curl**: Command-line HTTP requests
3. **Python requests**: Programmatic data collection
4. **Error handling**: Timeouts, retries, status codes
5. **OMDb API**: Real-world movie data
6. **BeautifulSoup**: Web scraping when APIs don't exist
7. **Data pipelines**: Building reusable collection code

## Next Week

**Week 2: Data Validation & Quality**

The data we collected today is messy! Next week we'll learn:
- Schema validation with Pydantic
- Data type cleaning
- Handling missing values
- Quality metrics

---

## Submission

Save your completed notebook and submit:
1. This notebook with all cells executed
2. The CSV file of movies you collected
3. A brief summary (1 paragraph) of what you learned